In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -r /content/drive/MyDrive/RAG-LangChain/requirements.txt --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 193.4 kB/s eta 0:00:00
Reason for being yanked: The Windows wheels, under some conditions, caused segfaults in unrelated user code.  Due to this we deleted the Windows wheels to prevent these segfaults, however this caused greater disruption as pip then began to try (and fail) to build 3.9.1 from the sdist on Windows which impacted far more users.  Yanking the whole release is the only tool available to eliminate these failures without changes to on the user side.  The sdist, OSX wheel, and manylinux wheels are all functional and there are no critical bugs in the release.   Downstream packagers sho

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [3]:
origem_db_config = {
    "host": "dpg-cs2rgj68ii6s738b6dpg-a.oregon-postgres.render.com",
    "port": "5432",
    "dbname": "bd_semana_academica",
    "user": "admin",
    "password": "h55P9iPCzO1KypcVVQ3vJofdKCPQBGhL"
}
origem_con_string = f"postgresql://{origem_db_config['user']}:{origem_db_config['password']}@{origem_db_config['host']}:{origem_db_config['port']}/{origem_db_config['dbname']}"
engine_origem = create_engine(origem_con_string)

In [4]:
query = """
    select codigo_produto, descricao_produto, preco_produto, quantidade_estoque, dt_vencimento
    from public.produto;
    """

# Extrair dados usando pandas
df = pd.read_sql(query, engine_origem)

In [5]:
#def summary(tabela):
#    txt = f'''Produto {tabela['descricao_produto']} codigo {tabela['codigo_produto']} tem o preco de {tabela['preco_produto']}
#    tem {tabela['quantidade_estoque']} de estoque e tem validade ate a data {tabela['dt_vencimento']}'''
#
#    return txt

def summary(tabela):
    txt = f'''Descricao: {tabela['descricao_produto']} Codigo: {tabela['codigo_produto']} Preço: {tabela['preco_produto']}
    Quantidade Estoque: {tabela['quantidade_estoque']} Validade: {tabela['dt_vencimento']}'''

    return txt

In [6]:
df['summary'] = df.apply(lambda df: summary(df), axis=1)

In [7]:
print(df)

    codigo_produto      descricao_produto  preco_produto  quantidade_estoque  \
0                1         Arroz Integral          12.99               150.0   
1                2           Feijão Preto           8.50               300.0   
2                3     Macarrão Espaguete           6.75               200.0   
3                4   Café Torrado e Moído          17.90               100.0   
4                5  Leite em Pó Desnatado          22.99                50.0   
..             ...                    ...            ...                 ...   
69              70         Whisky 12 Anos         189.99                40.0   
70              71        Vodka Importada          89.50                70.0   
71              72             Rum Branco          65.75                60.0   
72              73           Tequila Ouro         105.90                45.0   
73              74            Gin Premium         149.99                35.0   

   dt_vencimento                       

In [8]:
lista = df['summary'].tolist()

In [10]:
import yaml
import os
with open('/content/drive/MyDrive/RAG-LangChain/config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['PINECONE_API_KEY'] = config['PINECONE_API_KEY']
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone as PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from pinecone import Pinecone
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
index_name = 'dict'
vector_store = PineconeVectorStore.from_texts(lista, embeddings, index_name=index_name)

In [16]:
query_1 = '''Responda apenas com base no input fornecido. Com base na data de hoje, que é 12/10/2024, qual o proximo produto a vencer'''
llm = ChatOpenAI(model='gpt-4-turbo', temperature=0.2)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 50})
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)
answer_1 = chain.invoke(query_1)
print(retriever)

tags=['Pinecone', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7d9487fc1360> search_kwargs={'k': 50}


In [17]:
print('Pergunta: ',answer_1['query'])
print('Resultado: ',answer_1['result'],'\n')

Pergunta:  Responda apenas com base no input fornecido. Com base na data de hoje, que é 12/10/2024, qual o proximo produto a vencer
Resultado:  O próximo produto a vencer, com base na data de hoje (12/10/2024), é o Iogurte Natural, que tem validade em 20/10/2024. 

